### Применение линейной регрессии для построения предсказательной модели стоимости автомобилей

In [ ]:
import pandas as pd

In [35]:
df=pd.read_csv('cars.csv')

In [36]:
df['company']=df.CarName.apply(lambda c: c.split(' ')[0])

In [4]:
df.company.nunique()

28

In [5]:
df.drop('car_ID', axis='columns', inplace=True)
df.drop('CarName',axis='columns', inplace=True)

In [6]:
df.company=df.company.str.lower()

In [7]:
def fix_company(to_rep, how_rep):
    df.company.replace(to_rep, how_rep, inplace=True)

In [8]:
fix_company('maxda','mazda')
fix_company('porcshce','porsche')
fix_company('toyouta','toyota')
fix_company('vokswagen','volkswagen')
fix_company('vw','volkswagen')

In [9]:
df.company.nunique()

22

In [10]:
# Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:
cols_to_keep=['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower','price']

In [11]:
# отфильтруем датафрейм по указанным колонкам
df_cor=df[cols_to_keep]

In [12]:
# посчитаем корреляцию между price и другими переменными
df_cor.corr()['price'].round(2)

wheelbase     0.58
carlength     0.68
carwidth      0.76
curbweight    0.84
enginesize    0.87
boreratio     0.55
horsepower    0.81
price         1.00
Name: price, dtype: float64

In [13]:
df_cor.dtypes

company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
price             float64
dtype: object

In [14]:
# Последний шаг в подготовке данных: 
# линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), 
# поэтому применяем pd.get_dummies() 

In [15]:
df_dummy = pd.get_dummies(data=df_cor[['company','fueltype','aspiration','carbody','drivewheel','enginetype','cylindernumber']], drop_first = True)

In [16]:
cars_lr=pd.concat([df_cor.drop(['company','fueltype','aspiration','carbody','drivewheel','enginetype','cylindernumber'], axis='columns'),df_dummy],axis=1)

In [17]:
cars_lr.shape

(205, 49)

In [18]:
# Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

In [19]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [20]:
results=smf.ols('price~horsepower',cars_lr).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Fri, 30 Sep 2022   Prob (F-statistic):           1.48e-48
Time:                        15:51:23   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

In [21]:
# Теперь – две модели:
# 1) модель со всеми предикторами

In [22]:
X=cars_lr.drop(['price'], axis='columns')
X=sm.add_constant(X)
y=cars_lr['price']

C:\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [23]:
model=sm.OLS(y,X)
results=model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Fri, 30 Sep 2022   Prob (F-statistic):           4.86e-89
Time:                        15:51:24   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

In [24]:
# 2) модель со всеми предикторами, кроме марок машин

In [25]:
cars_lr.columns

Index(['wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginesize',
       'boreratio', 'horsepower', 'price', 'company_audi', 'company_bmw',
       'company_buick', 'company_chevrolet', 'company_dodge', 'company_honda',
       'company_isuzu', 'company_jaguar', 'company_mazda', 'company_mercury',
       'company_mitsubishi', 'company_nissan', 'company_peugeot',
       'company_plymouth', 'company_porsche', 'company_renault',
       'company_saab', 'company_subaru', 'company_toyota',
       'company_volkswagen', 'company_volvo', 'fueltype_gas',
       'aspiration_turbo', 'carbody_hardtop', 'carbody_hatchback',
       'carbody_sedan', 'carbody_wagon', 'drivewheel_fwd', 'drivewheel_rwd',
       'enginetype_dohcv', 'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf',
       'enginetype_ohcv', 'enginetype_rotor', 'cylindernumber_five',
       'cylindernumber_four', 'cylindernumber_six', 'cylindernumber_three',
       'cylindernumber_twelve', 'cylindernumber_two'],
      dtype='object'

In [26]:
# марки машин можно убрать с помощью .drop([перечисление колонок через запятую],axis='columns') 
# или через вот такую конструкцию с тильдой [~cars_lr.columns.str.startswith('company_')]
cols_to_keep=cars_lr.columns[~cars_lr.columns.str.startswith('company_')]

In [27]:
X=cars_lr[cols_to_keep].drop('price', axis='columns')
X=sm.add_constant(X)
y=cars_lr['price']

C:\Anaconda\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [28]:
model_2=sm.OLS(y,X)
results_2=model_2.fit()
print(results_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Fri, 30 Sep 2022   Prob (F-statistic):           9.86e-81
Time:                        15:51:24   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

### Выбранная модель объясняет примерно 90 % дисперсии (окр. до целого). Среди предикторов 10 из 27 оказались не значимыми (p > 0.05). При единичном изменении показателя horsepower, цена ВОЗРАСТАЕТ на 86.8164